In [ ]:
import numpy as np

class nn:
    def __init__(self):
        self.bias_weights = []
        self.layer_weights = []
        self.activation_funcs = []
        self.backward_activation_funcs = []
        self.learning_rate = 0.1
        self.train_costs = []
        self.test_costs = []
        self.train_accuracies = []
        self.test_accuracies = []

    def initialise_weights(self, input_size, nodes):
        """
        Set up our weight matrices (bias and layers)
        param input_size: Number of features in our input data 
        param nodes: A list of integers that determines the number of layers and nodes in each
        """
        nodes.insert(0, input_size)
        # See paper 1 for how weights are indexed
        self.layer_weights=[]
        self.bias_weights=[]
        # Create layers
        for i in range(1, len(nodes)):
            self.layer_weights.append(np.random.normal(0,0.1,size=(nodes[i-1],nodes[i])))
            self.bias_weights.append(np.zeros((1, nodes[i])))

    def initialise_activations(self, activation_func_names):
        """
        Set up activation functions for use in forward and backward pass
        param activation_func_names: Which activation functions as a list of strings
        """
        for name in activation_func_names:
            if name == 'relu':
                self.activation_funcs.append(relu)
                self.backward_activation_funcs.append(backward_relu)
            elif name == 'sigmoid':
                self.activation_funcs.append(sigmoid)
                self.backward_activation_funcs.append(sigmoid_backward)
            else:
                raise Exception("Activation function is not implemented. Must be either relu or sigmoid.")
    
    def use_weights(self, x):
        """
        Use the current weights to generate the output of the last layer without any other functions applied
        param x: 2d array of inputs to use the weights on
        """
        num_layers = len(self.layer_weights)
        output = x@self.layer_weights[0]
        output = self.activation_funcs[0](output)
        for i in range(1,num_layers):
            output = output@self.layer_weights[i] + self.bias_weights[i]
            output = self.activation_funcs[i](output)
        return output
    
    def model_forward(self, x):
        """
        Use the current weights to generate values for each layer (transfer) and activation
        param x: 2d array of inputs to use the weights on
        return: A list of the transfer function outputs, A list of the activation function outputs
        """
        num_layers = len(self.layer_weights)
        transfers = []
        activations = []
        transfers.append(x@self.layer_weights[0])
        activations.append(self.activation_funcs[0](transfers[0]))
        for i in range(1,num_layers):
            transfers.append((activations[-1]@self.layer_weights[i]) + self.bias_weights[i])
            activations.append(self.activation_funcs[i](transfers[-1]))
        return transfers, activations

    def model_backward(self, x, y, transfers, activations):
        """
        Compute the gradients for each weights (bias and layers) with respect to the cost function (Softmax-Cross entropy)
        param x: 2d array of inputs to use the weights on
        param y: 2d array of true values
        param tansfers: array of outputs from transfer layers in the neural network
        param activations: array of outputs from activation layers in neural network
        return: List of weight gradients, List of bias gradients
        """
        weight_grads = []
        bias_grads = []
        
        d_s = backward_softmax_cross(activations[-1], y) #Derivative of cross entropy and softmax
        d_prev_layer = self.backward_activation_funcs[-1](d_s, transfers[-1]) #Go through last activation
        num_layers = len(self.layer_weights)
        num_samples = len(y)
        
        #Go through each layer in the network
        for i in reversed(range(0, num_layers)):
            if (i==0):
                #The final layer so use the inputs
                previous_vals = x
            else:
                #Not the final layer so use the activations from the previous activations
                previous_vals = activations[i-1]
           # Gradient for the current layer weights, See paper 1 and 2
            weight_grad = np.divide((previous_vals.T@d_prev_layer),num_samples)
            weight_grads.append(weight_grad)
            # Bias for the current layer bias. 
            # The sum of the gradients of the previous layer as derivative is just 1 and then need to sum previous grads
            bias_grad = np.divide(np.sum(d_prev_layer ,axis=0),num_samples)
            bias_grads.append(bias_grad)

            #If we are not at the last layer we need to compute the gradients to flow backward
            if (i!=0):  
                # Retrieve the current weights and use them to compute the next set of gradients
                # See paper 3
                current_layer = self.layer_weights[i]
                d_r = d_prev_layer@current_layer.T
                d_prev_layer = self.backward_activation_funcs[i-1](d_r, transfers[i-1])
        # Return the reversed for easier iterating they are now in the order of the layers forward
        weight_grads.reverse()
        bias_grads.reverse()
        return weight_grads, bias_grads

    def update_parameters(self, weight_grads, bias_grads):
        """
        Use the computed gradients and learning rate from model_backward to update the weights
        param weight_grads: list of weight gradients 
        param bias_grads: list of bias gradients
        """
        num_layers = len(self.layer_weights)
        for i in range(0, num_layers):  
            self.layer_weights[i] -= weight_grads[i] * self.learning_rate
            self.bias_weights[i] -= bias_grads[i] * self.learning_rate
        
    def step(self, x, y):
        """
        Helper function to make an iteration of gradient descent, with a forward pass, backward pass and update
        param x: input values for determining gradients
        param y: true values for determining gradients
        """
        transfers, activations = self.model_forward(x)
        weight_gradients, bias_gradients = self.model_backward(x, y, transfers, activations)
        self.update_parameters(weight_gradients, bias_gradients)
                        
    def train(self, x_train, y_train, nodes, activation_funcs, learning_rate=0.1, epochs=5, batch_size=32, x_test = [], y_test = []):
        """
        Trains the given neural network on the inputted data using mini batching
        param x_train: 2d numpy array of training data x values
        param y_train: 2d numpy array of true values for training set (one-hot encoded)
        param nodes: array of layer nodes (Final layer must match length of one-hot encoded output values)
        param activation_funcs: array of strings indicating activation functions that should be used
        param learning_rate: gradient multiplier in gradient descent
        param epochs: number of times to go through entire x_train set
        param batch_size: size of mini-batches
        param x_test: 2d numpy array of test data x values
        param y_test: 2d numpy array of true values for test set (one-hot encoded)
        """
        self.initialise_weights(len(x_train[0]), nodes)
        self.initialise_activations(activation_funcs)
        self.learning_rate = learning_rate
        epochs_no = 0
        iter_no = 0
        while epochs>epochs_no:
            for x_batch, y_batch in self.batch_generator(x_train,y_train,batch_size):
                self.step(x_batch, y_batch)
                self.record(x_batch, y_batch, x_test, y_test, iter_no) # This dramatically increases runtime
                if self.test_accuracies[-1] >= 0.90:
                    break
                iter_no += 1
            if self.test_accuracies[-1] >= 0.90:
                break
            epochs_no += 1

    def batch_generator(self, X, y, batchsize):
        """
        Generator function to return mini batches of X and Y
        param X: One list to create mini batches on
        param Y: One list to create mini batches on
        return: A mini batch of size batchsize of X, A mini batch of size batchsize of y
        """
        p = np.random.permutation(len(y))
        shuffled_X = X[p]
        shuffled_Y = y[p]
        for start_index in range(0, len(X) - batchsize + 1, batchsize):
            batch = slice(start_index, start_index + batchsize)
            yield shuffled_X[batch], shuffled_Y[batch]

    def record(self,x_train, y_train, x_test, y_test, iter_no):
        """
        Records the train and test cost and accuracy at each iteration and prints based on iteration print
        param x_train: Training values used this iteration
        param y_train: True values used this iteration
        param x_test: Test values
        param y_test: True values for test values
        param iter_no: Current iteration of steps
        """
        ITERATION_PRINT = 500 
        train_cost = self.cost(x_train, y_train)
        self.train_costs.append(train_cost)
        test_cost = self.cost(x_test, y_test)
        self.test_costs.append(test_cost)
        train_acc = self.accuracy(x_train, y_train)
        self.train_accuracies.append(train_acc)
        test_acc = self.accuracy(x_test, y_test)
        self.test_accuracies.append(test_acc)
        if iter_no%ITERATION_PRINT==0:
            print(f"Iteration {iter_no} - Train {train_cost} - Test {test_cost} - Train {train_acc} - Test {test_acc}")
    
    def accuracy(self, x, true_values):
        """
        Makes a prediction for the given values and returns the accuracy of the prediction
        param x: values to predict on
        param true_values: true values for given values (one-hot encoded)
        return: Accuracy of predictions
        """
        predictions = self.predict(x)
        return np.mean(true_values == predictions)

    def cost(self, x, true_values):
        """
        Makes a forward pass for the given values and returns the cost
        param x: values to predict on
        param true_values: true values for given values (one-hot encoded)
        return: Softmaxed cross-entropy loss of predictions
        """
        output = self.use_weights(x)
        return np.mean(softmax_cross_loss_all(output, true_values))
    
    def predict(self, x):
        """
        Makes a forward pass for the given values and takes the maximum as the prediction
        param x: values to predict on
        return: One-hot encoded prediction of x-value (Predicted class) 
        """
        output = self.use_weights(x)
        max_indices = np.argmax(output, axis=1)
        preds = np.zeros_like(output)
        rows = np.arange(preds.shape[0])
        preds[rows, max_indices] = 1
        return preds


def softmax_cross_loss_all(outputs, one_hot_true_labels):
    """
    Computes the softmax and cross entropy of values given their true labels (one-hot encoded)
    param outputs: 2d numpy array of values 
    param one_hot_true_labels: True labels of given outputs (one-hot encoded)
    return: Sum of costs for each output
    """
    # True class - log( Sum of exponents of all)
    row_max = np.max(outputs, axis=1, keepdims=True)
    maxs = np.repeat(row_max, outputs.shape[1], axis=1)
    mask = outputs > 0
    outputs[mask] -= maxs[mask]
    return -np.sum(np.multiply(outputs, one_hot_true_labels), axis=1) + np.log(np.sum(np.exp(outputs), axis=1))

def softmax_all(outputs):
    """
    Computes the softmax of values (Numerical stability by subtracting maximums if > 0)
    param outputs: 2d numpy array of values 
    return: 2d numpy array of softmax values
    """
    row_max = np.max(outputs, axis=1, keepdims=True)
    maxs = np.repeat(row_max, outputs.shape[1], axis=1)
    mask = outputs > 0
    outputs[mask] -= maxs[mask]
    exps = np.exp(outputs)
    sum_exps = np.sum(exps, axis=1, keepdims=True)
    return exps / sum_exps
                                                        
def backward_softmax_cross(outputs, one_hot_true_labels):
    """
    Computes derivative of the joint softmax and cross entropy function
    param outputs: 2d numpy array of values 
    param one_hot_true_labels: 2d array of true labels (one-hot encoded)
    return: 2d numpy array of derivatives
    """
    num_samples = one_hot_true_labels.shape[0]
    softmaxes = softmax_all(outputs)
    return (-one_hot_true_labels + softmaxes) / num_samples

def relu(inputs):
    """
    Applies the RELU function
    param inputs: 2d numpy array of values 
    return: 2d numpy array of values with RELU applied
    """
    return np.maximum(inputs,0)

def backward_relu(grads, inputs):
    """
    Computes the derivative of the RELU function
    param grads: 2d numpy array of values flowing backward through the RELU 
    param inputs: 2d numpy array of values that were inputs to the RELU function
    return: 2d numpy array of derivative of RELU function
    """
    grads[inputs <= 0] = 0;
    return grads;

def sigmoid(inputs):
    """
    Applies the sigmoid function
    param inputs: 2d numpy array of values 
    return: 2d numpy array of values with sigmoid applied
    """
    return np.divide(1,(1+np.exp(-inputs)))

def sigmoid_backward(grads, inputs):
    """
    Computes the derivative of the sigmoid function
    param grads: 2d numpy array of values flowing backward through the sigmoid 
    param inputs: 2d numpy array of values flowing backward through the sigmoid 
    return: 2d numpy array of derivative of sigmoid function
    """
    sigmoids = sigmoid(inputs)
    return sigmoids*(1-sigmoids)*grads

In [ ]:
from matplotlib import pyplot as plt
def training_curve_plot(title, train_losses, test_losses, train_accuracy, test_accuracy):
    """ convenience function for plotting train and test loss and accuracy
    """
    lg=13
    md=10
    sm=9
    fig, axs = plt.subplots(1, 2, figsize=(12, 4))
    fig.suptitle(title, fontsize=lg)
    x = range(1, len(train_losses)+1)
    axs[0].plot(x, test_losses, label=f'Final test loss: {test_losses[-1]:.4f}')
    axs[0].plot(x, train_losses, label=f'Final train loss: {train_losses[-1]:.4f}')
    axs[0].set_title('Losses', fontsize=md)
    axs[0].set_xlabel('Iteration', fontsize=md)
    axs[0].set_ylabel('Loss', fontsize=md)
    axs[0].legend(fontsize=sm)
    axs[0].tick_params(axis='both', labelsize=sm)
    # Optionally use a logarithmic y-scale
    #axs[0].set_yscale('log')
    axs[0].grid(True, which="both", linestyle='--', linewidth=0.5)
    axs[1].plot(x, 100*train_accuracy, label=f'Final train accuracy: {100*train_accuracy[-1]:.4f}%')
    axs[1].plot(x, 100*test_accuracy, label=f'Final test accuracy: {100*test_accuracy[-1]:.4f}%')
    axs[1].set_title('Accuracy', fontsize=md)
    axs[1].set_xlabel('Iteration', fontsize=md)
    axs[1].set_ylabel('Accuracy (%)', fontsize=sm)
    axs[1].legend(fontsize=sm)
    axs[1].tick_params(axis='both', labelsize=sm)
    axs[1].grid(True, which="both", linestyle='--', linewidth=0.5)
    plt.savefig('loss_acc.png')
    plt.show()

def plot_weights(weights):
    fig, axs = plt.subplots(2, 5, figsize=(10, 4))
    for i, ax in enumerate(axs.flat):
        ax.imshow(weights[i], cmap='Greys')
    fig.suptitle('Single layer neural network weights after training')
    plt.tight_layout()
    plt.savefig('weights.png')
    plt.show()

In [ ]:
from load_mnist import load_mnist
X_train, Y_train, X_test, Y_test = load_mnist()

In [ ]:
import time
the_nn = nn()
start_time = time.time()
the_nn.train(X_train, Y_train, [111, 10], ['relu','relu'], learning_rate=0.4, epochs=80, batch_size=50, x_test = X_test, y_test = Y_test)
training_time = time.time() - start_time
print(training_time)

In [ ]:
training_curve_plot("Multi-layer neural network training", np.array(the_nn.test_costs), np.array(the_nn.train_costs), np.array(the_nn.train_accuracies), np.array(the_nn.test_accuracies))

In [ ]:
the_nn_1 = nn()
the_nn_1.train(X_train, Y_train, [10], ['relu'], learning_rate=0.5, epochs=1, batch_size=40, x_test = X_test, y_test = Y_test)

In [ ]:
training_curve_plot("Single layer nn", np.array(the_nn_1.test_costs), np.array(the_nn_1.train_costs), np.array(the_nn_1.train_accuracies), np.array(the_nn_1.test_accuracies))

In [ ]:
weights = the_nn_1.layer_weights[-1].T.reshape(10,28,28)
plot_weights(weights)